In [41]:
import boto3

s3 = boto3.client("s3")
bucket = "ooi-rca-qaqc-prod"

In [44]:

def generate_data_availability(instrument, year):

    year = int(year)
    prefix = f"spectrograms/{year}/{instrument}/"

    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

    files = []
    for obj in response.get("Contents", []):
        files.append(obj["Key"])

    date_re = re.compile(r'(\d{8})\.png$')

    present = set()
    
    for f in files:
        m = date_re.search(f)
        if m:
            ymd = m.group(1)
            d = date(int(ymd[:4]), int(ymd[4:6]), int(ymd[6:8]))
            present.add(d)

    # build full year
    start, end = date(year,1,1), date(year+1,1,1)
    days = []
    cur = start
    while cur < end:
        days.append(cur)
        cur += timedelta(days=1)

    df = pd.DataFrame({
        "date": pd.to_datetime(days), 
        "has_data": [d in present for d in days],
        })

    return df

In [45]:
df = generate_data_availability('HYDBBA102', '2024')

In [46]:
df.head()

,date,has_data
0,2024-01-01,True
1,2024-01-02,True
2,2024-01-03,True
3,2024-01-04,True
4,2024-01-05,True
